<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/recommendation_system_for_old_and_new_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Step 1: Load the dataset

import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/zidan/aspect_sentiments.csv')

In [50]:
columns_to_drop = ['Price', 'Packaging', 'Delivery Cost', 'Quality', 'Overall Sentiment']
dataset.drop(columns_to_drop, axis=1, inplace=True)

In [51]:
dataset.head(10)

,Product_name,Review,Overall Sentiment Score
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0
5,"Salate Cordless Electric Spin Scrubber, Electr...",It has really made my life so much easy! Its v...,0
6,"Salate Cordless Electric Spin Scrubber, Electr...",So far I've used this powered scrubber a coupl...,1
7,"Salate Cordless Electric Spin Scrubber, Electr...",I did not like to clean the kitchen. This mach...,0
8,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Ladies and gentlemen, get ready for a review t...",0
9,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Oh, Ultra Strong Toilet Paper, how you do\nPro...",0


In [52]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Generate random reviewer names
def generate_reviewer_names(num_reviewers):
    first_names = ["John", "Jane", "Michael", "Emily", "David", "Olivia", "Daniel", "Sophia", "Matthew", "Emma", "Zidan", "Thinesh", "Albert", "Vinoth", "Saini", "Karthi"]
    last_names = ["Smith", "Johnson", "Brown", "Taylor", "Anderson", "Williams", "Jones", "Davis", "Miller", "Wilson", "Maryam", "Soosai", "Christopher","Vithujan", "Kumar", "Bathri"]
    reviewer_names = []
    for _ in range(num_reviewers):
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        reviewer_names.append(f"{last_name} {first_name}")
    return reviewer_names

# Number of reviewers to generate
num_reviewers = 500

# Generate reviewer information
reviewer_names = generate_reviewer_names(num_reviewers)
reviewer_ages = np.random.randint(low=18, high=76, size=num_reviewers)
reviewer_genders = np.random.choice(["Male", "Female"], size=num_reviewers)
reviewer_areas = np.random.choice(["Central", "Eastern", "Northern", "North Central", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"], size=num_reviewers)

# Create a dictionary to map reviewer names with their respective age, gender, and area
reviewer_info = {name: (age, gender, area) for name, age, gender, area in zip(reviewer_names, reviewer_ages, reviewer_genders, reviewer_areas)}

# Assign random reviewer names to each row in the dataset
dataset['reviewer_name'] = random.choices(reviewer_names, k=len(dataset))

# Assign reviewer age, gender, and area based on the reviewer name
dataset['age'], dataset['gender'], dataset['area'] = zip(*[reviewer_info[name] for name in dataset['reviewer_name']])

# Extract the features
reviews = dataset["Review"]
product_names = dataset["Product_name"]
overall_sentiment_score = dataset["Overall Sentiment Score"]
reviewer_name = dataset["reviewer_name"]
age = dataset["age"]
gender = dataset["gender"]
area = dataset["area"]

In [53]:
dataset

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0,Anderson Michael,43,Male,Central
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1,Taylor Jane,67,Male,Western
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2,Johnson Vinoth,20,Female,Western
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0,Davis Matthew,21,Male,Northern
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0,Maryam Emily,29,Female,Northern
...,...,...,...,...,...,...,...
11052,2 Pack Pumice Stone for Toilet Cleaning with E...,Work great,0,Maryam Emma,42,Male,Eastern
11053,2 Pack Pumice Stone for Toilet Cleaning with E...,The tool works fairly well for removing hard s...,4,Jones Karthi,66,Male,Eastern
11054,2 Pack Pumice Stone for Toilet Cleaning with E...,Long handle so you don’t have to put your hand...,1,Anderson Michael,43,Male,Central
11055,2 Pack Pumice Stone for Toilet Cleaning with E...,It works very nicely on these Arizona hard wat...,1,Jones David,56,Male,Central


In [54]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the reviews into TF-IDF vectors
review_vectors = vectorizer.fit_transform(reviews)

# Calculate the cosine similarity between all pairs of products
product_similarity = cosine_similarity(review_vectors)

# Create a function to recommend products to a user
def recommend_products(user_age, user_gender, user_area):
    # Validate age input
    if user_age <= 18 or user_age >= 75:
        raise ValueError("Age should be between 18 and 75.")
        
    # Validate gender input
    valid_genders = ["Male", "Female"]
    if user_gender not in valid_genders:
        raise ValueError("Invalid gender. Please enter 'Male' or 'Female' or 'Other'.")
        
    # Validate area input
    valid_areas = ["Central", "Eastern", "Northern", "North Central", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"]
    if user_area not in valid_areas:
        raise ValueError("Invalid area. Please enter a valid province name.")

    # Filter products that match the user's age, gender, and area
    filtered_indices = np.where((age == user_age) & (gender == user_gender) & (area == user_area))[0]

    # Calculate the similarity scores for the filtered products
    similarity_scores = np.mean(product_similarity[filtered_indices], axis=0)

    # Sort the products by their similarity to the user
    similar_products = np.argsort(similarity_scores)[::-1]

    # Initialize set to keep track of recommended product names
    recommended_set = set()

    # Return the top 10 unique products
    recommendations = []
    for product in similar_products:
        if len(recommendations) >= 10:
            break
        product_name = product_names[product]
        if product_name not in recommended_set:
            recommendations.append(product_name)
            recommended_set.add(product_name)
    return recommendations

# Recommend products to a user
user_age = 34
user_gender = "Female"
user_area = "Western"

try:
    recommended_products = recommend_products(user_age, user_gender, user_area)
    # Print the recommended products with rank numbers
    for rank, product in enumerate(recommended_products, start=1):
        print(f"Rank {rank}: {product}")
except ValueError as e:
    print("Error:", str(e))

Rank 1: LiBa Electric Bug Zapper, Indoor Insect Killer - (2) Extra Replacement Bulbs - Fly, Mosquito Killer and Repellent - Lightweight, Powerful 2800V Grid, Easy-to-Clean, with a Removable Washable Tray.
Rank 2: Flowtron BK-80D 80-Watt Electronic Insect Killer, 1-1/2 Acre Coverage , Black
Rank 3: SUPERBIO 2.6 Gallon Compostable Handle Tie Garbage Bags, 100 Count, 1 Pack, Kitchen Trash Bag With Handle, Food Scrap Small Bags Certified by BPI and OK Compost Meeting ASTM D6400 Standards
Rank 4: Leather Honey Leather Conditioner, Best Leather Conditioner Since 1968. for Use on Leather Apparel, Furniture, Auto Interiors, Shoes, Bags and Accessories. Non-Toxic and Made in The USA!…
Rank 5: Angel Soft Toilet Paper with Fresh Lavender Scent, 48 Mega Rolls = 192 Regular Rolls, 2-Ply Bath Tissue
Rank 6: OXO Good Grips Dustpan and Brush Set, White
Rank 7: Poo-Pourri Before-You-Go Toilet Spray Bottle, Original Scent, 2 Fl Oz (Pack of 2)
Rank 8: Quilted Northern Ultra Soft & Strong Toilet Paper wit

In [55]:
dataset

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0,Anderson Michael,43,Male,Central
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1,Taylor Jane,67,Male,Western
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2,Johnson Vinoth,20,Female,Western
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0,Davis Matthew,21,Male,Northern
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0,Maryam Emily,29,Female,Northern
...,...,...,...,...,...,...,...
11052,2 Pack Pumice Stone for Toilet Cleaning with E...,Work great,0,Maryam Emma,42,Male,Eastern
11053,2 Pack Pumice Stone for Toilet Cleaning with E...,The tool works fairly well for removing hard s...,4,Jones Karthi,66,Male,Eastern
11054,2 Pack Pumice Stone for Toilet Cleaning with E...,Long handle so you don’t have to put your hand...,1,Anderson Michael,43,Male,Central
11055,2 Pack Pumice Stone for Toilet Cleaning with E...,It works very nicely on these Arizona hard wat...,1,Jones David,56,Male,Central


In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Filter the data for the old user
old_user = 'Johnson Sophia'
old_user_data = dataset[dataset['reviewer_name'] == old_user]

# Filter the data for products with sentiment score less than 2
filtered_data = dataset[dataset['Overall Sentiment Score'] < 2]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Apply TF-IDF vectorization to the review text
review_vectors = vectorizer.fit_transform(dataset['Review'])

# Calculate cosine similarity between the old user's reviews and all products
cosine_similarities = cosine_similarity(vectorizer.transform(old_user_data['Review']), review_vectors)

# Get the average sentiment score for each product
dataset['Average_Sentiment'] = dataset.groupby('Product_name')['Overall Sentiment Score'].transform('mean')

# Combine the average sentiment scores with the cosine similarities
dataset['Score'] = cosine_similarities[0] * dataset['Average_Sentiment']

# Sort the data based on the score
sorted_data = dataset.sort_values('Score', ascending=False)

# Get the top 10 recommended products
top_10_products = sorted_data['Product_name'].unique()[:10]

print("Top 10 recommended products for", old_user, ":")
print(top_10_products)


Top 10 recommended products for Johnson Sophia :
['HARRIS Diatomaceous Earth Food Grade, 4lb with Powder Duster Included in The Bag'
 'Compact Coreless 2-Ply Recycled Toilet Paper by GP PRO (Georgia-Pacific), 19375, 1,000 Sheets Per Roll, 36 Rolls Per Case'
 'Hefty Load & Carry Heavy Duty Contractor Large Trash Bags, 42 Gallon, 26 Count'
 'Jawlark Bug Zapper Outdoor, 4200V Electric Mosquito Zapper Indoor, Insect Fly Zapper Waterproof with 5ft Power Cord, Mosquito Killer for Home, Patio, Kitchen, Backyard, Camping, Plug-in'
 'Air Wick Plug in Scented Oil Refill, 10ct, Summer Delights, Scented Oil, Air Freshener, Essential Oils, Eco Friendly'
 'DynaTrap 41050 UV Replacement Bulb for DynaTrap Mosquito & Flying Insect Trap Models DT1050, DT1100, DT1260, DT250IN, DT300IN, DT1000-12V, DT1125, DT1200, DT1210 and DT1250'
 'BLACK+DECKER Bug Zapper, Electric UV Insect Catcher & Killer for Flies, Mosquitoes, Gnats & Other Small to Large Flying Pests, 1 Acre Outdoor Coverage for Home, Deck, Garden

In [57]:
john_data = dataset[dataset['reviewer_name'] == 'Johnson Sophia']

# Print the data for reviewer name "Johnson Sophia"
john_data

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area,Average_Sentiment,Score
67,"Hefty Ultra Strong Tall Kitchen Trash Bags, La...",This is a great trash bag. Love the scent bec...,0,Johnson Sophia,44,Male,Southern,0.000,0.000000
279,Swiffer Sweeper Wet Mopping Cloth Multi Surfac...,I am a single guy and want to keep things as s...,0,Johnson Sophia,44,Male,Southern,0.375,0.017764
331,Amazon Basics LR44 Alkaline Button Coin Cell B...,These are good batteries that last quite a whi...,0,Johnson Sophia,44,Male,Southern,0.750,0.038042
486,"Amazon Basics Freezer Gallon Bags, 90 Count (P...",I recently purchased the Amazon Basic Quart Si...,0,Johnson Sophia,44,Male,Southern,1.000,0.086734
529,"Scott® Multifold Paper Towels (01840), with Ab...",When I opened the box I was shocked at the siz...,1,Johnson Sophia,44,Male,Southern,0.875,0.083073
...,...,...,...,...,...,...,...,...,...
10326,John Boos Block MYSCRM Essential Mystery Oil a...,Great product that really nourishes and protec...,0,Johnson Sophia,44,Male,Southern,0.500,0.030078
10716,"[100 Pack] 3 oz. Clear Plastic Cups, Small Dis...",Smaller than I thought. But worked well!,1,Johnson Sophia,44,Male,Southern,0.375,0.020268
10826,"Small Kitchen Towels Dish Towels, 6 x 10 inch,...",I love these dish clothes can use for several ...,0,Johnson Sophia,44,Male,Southern,0.250,0.016331
10846,"akgk Nitrile Gloves, 100 Pcs Disposable Black ...",yep,0,Johnson Sophia,44,Male,Southern,1.500,0.000000
